In [3]:
import requests
import pandas as pd
import time
import schedule
import smtplib
import matplotlib.pyplot as plt
import sqlite3
from email.mime.text import MIMEText
from datetime import datetime
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

API_KEY = '3944fa7e7bafd4a987ee692c7f850cdf'  # Replace with your actual API key
CITIES = ['Delhi', 'Mumbai', 'Chennai', 'Bangalore', 'Kolkata', 'Hyderabad']
BASE_URL = 'http://api.openweathermap.org/data/2.5/weather'
ALERT_THRESHOLD = 35.0
ALERT_EMAIL = 'ishandewangan12@gmail.com'
USER_EMAIL = 'ishandewangan12@gmail.com'
USER_PASSWORD = 'umzi fnns woau wzqp'
UPDATE_INTERVAL = 5  # Minutes
CONSECUTIVE_ALERTS = 2

# Global Variables
recent_temperatures = []
alert_sent_time = None
weather_data = []
triggered_alerts = []

# Initialize SQLite Database
def init_db():
    try:
        conn = sqlite3.connect('weather_summary.db')
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS daily_summary (
                date TEXT PRIMARY KEY,
                average_temp REAL,
                max_temp REAL,
                min_temp REAL,
                humidity REAL,
                wind_speed REAL,
                dominant_condition TEXT
            )
        ''')
        conn.commit()
    except sqlite3.Error as e:
        logging.error(f"Database error during initialization: {e}")
    finally:
        conn.close()

# Fetch Weather Data
def fetch_weather(city):
    try:
        params = {
            'q': city,
            'appid': API_KEY,
            'units': 'metric'
        }
        logging.info(f"Requesting weather data for {city}.")
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json()

        main = data['main']
        wind = data['wind']
        weather = data['weather'][0]
        return {
            'city': city,
            'temp': main['temp'],
            'humidity': main['humidity'],
            'wind_speed': wind['speed'],
            'condition': weather['main'],
            'dt': datetime.fromtimestamp(data['dt']),
        }
    except requests.exceptions.HTTPError as e:
        logging.error(f"HTTP error while fetching weather for {city}: {e}")
    except KeyError as e:
        logging.error(f"Key error while processing weather data for {city}: {e}")
    except Exception as e:
        logging.error(f"An error occurred while fetching weather data for {city}: {e}")
    return None

# Store Weather Data
def store_weather_data(data):
    weather_data.append(data)

# Store Daily Summary
def store_daily_summary(daily_summary):
    try:
        conn = sqlite3.connect('weather_summary.db')
        cursor = conn.cursor()
        for _, row in daily_summary.iterrows():
            cursor.execute('''
                INSERT OR REPLACE INTO daily_summary (date, average_temp, max_temp, min_temp, humidity, wind_speed, dominant_condition)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (row['date'], row['average_temp'], row['max_temp'], row['min_temp'], row['humidity'], row['wind_speed'], row['dominant_condition']))
        conn.commit()
    except sqlite3.Error as e:
        logging.error(f"Database error while storing daily summary: {e}")
    except Exception as e:
        logging.error(f"An error occurred while storing daily summary: {e}")
    finally:
        conn.close()

# Calculate Daily Summary
def calculate_daily_summary():
    if not weather_data:
        logging.warning("No weather data available to calculate summary.")
        return

    try:
        df = pd.DataFrame(weather_data)
        df['date'] = df['dt'].dt.date

        daily_summary = df.groupby('date').agg(
            average_temp=('temp', 'mean'),
            max_temp=('temp', 'max'),
            min_temp=('temp', 'min'),
            humidity=('humidity', 'mean'),
            wind_speed=('wind_speed', 'mean'),
            dominant_condition=('condition', lambda x: x.mode()[0])
        ).reset_index()

        logging.info("Daily Weather Summary:")
        for index, row in daily_summary.iterrows():
            logging.info(f"Date: {row['date']}, Average Temp: {row['average_temp']}°C, "
                         f"Max Temp: {row['max_temp']}°C, Min Temp: {row['min_temp']}°C, "
                         f"Humidity: {row['humidity']}%, Wind Speed: {row['wind_speed']} m/s, "
                         f"Dominant Condition: {row['dominant_condition']}")

        store_daily_summary(daily_summary)
        visualize_daily_summary(daily_summary)
        display_historical_trends()
        visualize_triggered_alerts()

    except Exception as e:
        logging.error(f"An error occurred while calculating daily summary: {e}")

# Check Alerts
def check_alerts(current_temp):
    global alert_sent_time, recent_temperatures

    if not isinstance(current_temp, (int, float)):
        logging.warning("Invalid temperature value. Must be a number.")
        return

    recent_temperatures.append(current_temp)
    if len(recent_temperatures) > CONSECUTIVE_ALERTS:
        recent_temperatures.pop(0)

    if len(recent_temperatures) == CONSECUTIVE_ALERTS and all(temp > ALERT_THRESHOLD for temp in recent_temperatures):
        current_time = datetime.now()
        if alert_sent_time is None or (current_time - alert_sent_time).total_seconds() / 60 >= UPDATE_INTERVAL:
            alert_message = f"Alert! The temperature has exceeded {ALERT_THRESHOLD}°C for {CONSECUTIVE_ALERTS} consecutive readings."
            send_alert(alert_message)
            triggered_alerts.append((current_time, current_temp))  # Store triggered alerts with timestamp
            alert_sent_time = current_time

# Send Alert
def send_alert(alert_message):
    msg = MIMEText(alert_message)
    msg['Subject'] = 'Weather Alert'
    msg['From'] = USER_EMAIL
    msg['To'] = ALERT_EMAIL

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(USER_EMAIL, USER_PASSWORD)
            server.sendmail(USER_EMAIL, ALERT_EMAIL, msg.as_string())
            logging.info(f"Alert sent to {ALERT_EMAIL}.")
    except Exception as e:
        logging.error(f"Failed to send alert: {e}")

# Visualize Daily Summary
def visualize_daily_summary(daily_summary):
    plt.figure(figsize=(14, 8))

    # Plotting average, max, and min temperatures
    plt.plot(daily_summary['date'], daily_summary['average_temp'], label='Average Temp (°C)', marker='o', color='blue')
    plt.plot(daily_summary['date'], daily_summary['max_temp'], label='Max Temp (°C)', marker='o', color='red')
    plt.plot(daily_summary['date'], daily_summary['min_temp'], label='Min Temp (°C)', marker='o', color='green')

    # Adding humidity and wind speed
    plt.twinx()  # Create a second y-axis
    plt.plot(daily_summary['date'], daily_summary['humidity'], label='Humidity (%)', marker='x', color='purple', linestyle='--')
    plt.plot(daily_summary['date'], daily_summary['wind_speed'], label='Wind Speed (m/s)', marker='s', color='orange', linestyle=':')

    # Adding labels and styling
    plt.title('Daily Weather Summary (Temp, Humidity, Wind)', fontsize=16)
    plt.xlabel('Date', fontsize=14)
    plt.ylabel('Temperature (°C)', fontsize=14)
    plt.xticks(rotation=45, fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend(fontsize=12)
    plt.grid()
    plt.tight_layout()
    plt.savefig('daily_weather_summary.png')  # Save as an image file
    plt.show()  # Display the plot
    print("\nDaily Weather Summary:")
    for index, row in daily_summary.iterrows():
        summary_text = (f"Date: {row['date']}, Average Temp: {row['average_temp']}°C, "
                        f"Max Temp: {row['max_temp']}°C, Min Temp: {row['min_temp']}°C, "
                        f"Humidity: {row['humidity']}%, Wind Speed: {row['wind_speed']} m/s, "
                        f"Dominant Condition: {row['dominant_condition']}")
        print(summary_text)
        print("\n\n\n")

# Display Historical Trends
def display_historical_trends():
    try:
        conn = sqlite3.connect('weather_summary.db')
        df = pd.read_sql_query("SELECT * FROM daily_summary ORDER BY date", conn)
        conn.close()

        if df.empty:
            logging.warning("No historical data available to display.")
            return

        plt.figure(figsize=(14, 8))
        plt.plot(df['date'], df['average_temp'], label='Average Temp (°C)', marker='o', color='blue', linestyle='-')
        plt.plot(df['date'], df['max_temp'], label='Max Temp (°C)', marker='o', color='red', linestyle='-')
        plt.plot(df['date'], df['min_temp'], label='Min Temp (°C)', marker='o', color='green', linestyle='-')

        plt.title('Historical Temperature Trends', fontsize=16)
        plt.xlabel('Date', fontsize=14)
        plt.ylabel('Temperature (°C)', fontsize=14)
        plt.xticks(rotation=45, fontsize=12)
        plt.yticks(fontsize=12)
        plt.legend(fontsize=12)
        plt.grid()
        plt.tight_layout()
        plt.savefig('historical_temperature_trends.png')  # Save as an image file
        plt.show()  # Display the plot

    except Exception as e:
        logging.error(f"An error occurred while displaying historical trends: {e}")

# Visualize Triggered Alerts
def visualize_triggered_alerts():
    if not triggered_alerts:
        logging.warning("No triggered alerts to visualize.")
        return

    alert_times = [alert[0] for alert in triggered_alerts]
    alert_temps = [alert[1] for alert in triggered_alerts]

    plt.figure(figsize=(14, 6))
    plt.plot(alert_times, alert_temps, marker='o', color='orange', label='Triggered Alerts (°C)')
    plt.axhline(y=ALERT_THRESHOLD, color='red', linestyle='--', label='Alert Threshold (35°C)')

    plt.title('Triggered Alerts Over Time', fontsize=16)
    plt.xlabel('Time', fontsize=14)
    plt.ylabel('Temperature (°C)', fontsize=14)
    plt.xticks(rotation=45, fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend(fontsize=12)
    plt.grid()
    plt.tight_layout()
    plt.savefig('triggered_alerts.png')  # Save as an image file
    plt.show()  # Display the plot

# Job Function
def job():
    global weather_data
    weather_data = []  # Reset weather data for new updates

    for city in CITIES:
        weather = fetch_weather(city)
        if weather:
            store_weather_data(weather)
            check_alerts(weather['temp'])
        else:
            logging.warning(f"Weather data not fetched for {city}.")

    calculate_daily_summary()

# Initialize the database
init_db()

# Schedule the job
schedule.every(UPDATE_INTERVAL).minutes.do(job)

logging.info("Weather monitoring system started.")

while True:
    try:
        schedule.run_pending()
        time.sleep(1)
    except KeyboardInterrupt:
        logging.info("Weather monitoring system stopped by user.")
        break
    except Exception as e:
        logging.error(f"An error occurred during scheduling: {e}")
